In [1]:
from src.data.data_pipeline import DataPipeline
from src.models.model_factory import ModelFactory
from src.training.base_trainer import BaseTrainer

import os

In [2]:
import torch
import numpy as np
import random

In [3]:
SEED = 85
def seed_everything(seed):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_everything(SEED)

In [4]:
CONFIG_DIR = "/home/karan/Documents/GitHub/BTP/torchmanager/configs"
DATA = "data_no_aug.yaml"
MODEL = "model_svit3.yaml"
TRAINER = "training.yaml"

In [5]:
data_config_path = os.path.join(CONFIG_DIR, DATA)
data_pipeline = DataPipeline(data_config_path)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
train_dataloader, val_dataloader, test_dataloader = data_pipeline.data_loaders.values()

In [7]:
model_config_path = os.path.join(CONFIG_DIR, MODEL)
model_factory = ModelFactory(model_config_path)

In [8]:
model = model_factory.model

In [9]:
print(model)

SqueezeViT_3(
  (fire1): FireBlock(
    (squeeze_block): Sequential(
      (0): FireModule(
        (squeeze): ConvBlock(
          (conv_bn_act): Sequential(
            (0): Conv2d(3, 1, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): SiLU()
          )
        )
        (expand): ModuleDict(
          (1x1): ConvBlock(
            (conv_bn_act): Sequential(
              (0): Conv2d(1, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            )
          )
          (3x3): ConvBlock(
            (conv_bn_act): Sequential(
              (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            )
          )
        )
      )
    )
  )
  (fire2): FireBlock(
    (squeeze_block): Sequential(
      (0): FireModule(
        (squeeze): ConvBlock(
          (conv_bn_act): Sequential(
            (0): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): SiLU()
          )
        )
        (expand): ModuleDict

In [10]:
# print total parameters
total_params = sum(p.numel() for p in model.parameters())
print(f'{total_params:,} total parameters.')

534,317 total parameters.


In [11]:
class ClassifierTrainer(BaseTrainer):
    def __init__(self, train_config_path):
        super().__init__(train_config_path)
    
    def train_step(self, batch_data, batch_idx):
        # init_batch
        inputs, targets = batch_data

        # process_batch
        outputs = self.model(inputs)

        # compute_loss
        loss_value = self.loss(outputs, targets)

        # compute_metrics
        metrics_values = {}
        for metric_name, metric in self.metrics.items():
            metric_value = metric(outputs, targets)
            metrics_values[metric_name] = metric_value
        
        # end_batch
        results = {
            'loss' : loss_value,
            **metrics_values
        }

        return results
    
    def val_step(self, batch_data, batch_idx):
        return self.train_step(batch_data, batch_idx)
    
    def test_step(self, batch_data, batch_idx):
        return self.train_step(batch_data, batch_idx)

In [12]:
training_config_path = os.path.join(CONFIG_DIR, TRAINER)
training_manager = ClassifierTrainer(training_config_path)

In [13]:
# make dataloaders dict with only train and val
train_dataloaders = {
    'train': train_dataloader,
    'val': val_dataloader
}

train_history = training_manager.fit(model, train_dataloaders)

wandb: Currently logged in as: jain-49 (dl-codes). Use `wandb login --relogin` to force relogin


wandb: WARNING Source type is set to 'repo' but some required information is missing from the environment. A job will not be created from this run. See https://docs.wandb.ai/guides/launch/create-job


train/loss,█▄▃▂▁
train/multiLabelAccuracy,▁▁▁▁▁
val/loss,██▂▁▂
val/multiLabelAccuracy,▁▁▁▁▁
train/loss,1.77576
train/multiLabelAccuracy,0.0
val/loss,1.88914
val/multiLabelAccuracy,0.0


In [14]:
training_manager.test(model, test_dataloader)

{'test': {'loss': [tensor(1.8957, device='cuda:0')],
  'multiLabelAccuracy': [0.0]}}